 #  A Portfolio-Inflation Analyser
 ---

 In this project, we will use what we have learned over the past few weeks to evaluate the performance of 3 kinds of portfolios (a) Conservative (b) Growth and (c) Moderate over the last 10 years. We will evaluate which asset classes performed better and what was the impact of inflation. Lastly, we will provide recommendations on how you can rebalance your investments to reduce the effect of inflation. 
 Few points to keep in mind: 1. Use panda's to clean and format data sets 2. Describe the data exploration and cleanup  3. Create 6-8 visulaizations  4. Aggregate the visulaizations into a dashboard 5. Use an API and atleast one new python library
 

In [1]:
# Initial imports
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path
import seaborn as sns

%matplotlib inline


# Data Gathering

We used the Google Finance function to obtain the data for last 10 years for the following Tickers
and saved them as csv files (formatted Date to remove time component) in Resources Folder.

* VTSMX	Vanguard Total Stock Mkt Idx Inv: ' =GOOGLEFINANCE("VTSMX", "price", "1/1/2012", "12/31/2021", "DAILY") '
* VGTSX	Vanguard Total Intl Stock Index Inv: ' =GOOGLEFINANCE("VGTSX", "price", "1/1/2012", "12/31/2021", "DAILY") '
* VBMFX	Vanguard Total Bond Market Index Inv: ' =GOOGLEFINANCE("VBMFX", "price", "1/1/2017", "12/31/2021", "DAILY") '
* PIGLX	PIMCO Global Bond Opps (Unhedged): ' =GOOGLEFINANCE("PIGLX", "price", "1/1/2017", "12/31/2021", "DAILY")' '

Files:

* `VTSMX_data.csv`: Contains closing prices of Vanguard Total Stock Mkt Idx Inv

* `VGTSX_data.csv`: Contains closing prices of Vanguard Total Intl Stock Index Inv

* `VBMFX_data.csv`: Contains closing prices of Vanguard Total Bond Market Index Inv

* `PIGLX_data.csv`: Contains closing prices of PIMCO Global Bond Opps (Unhedged)

# Data Cleaning

In this section, we read the CSV files into DataFrames and perform any necessary data cleaning steps

## VTSMX Data
Read the VTSMX csv and clean the data.

In [2]:
# Set the Path
vtsmx_data = Path("Resources/VTSMX_data.csv")

# Read the historic closing prices and create a new daily returns DataFrame from the data. 
vtsmx_df = pd.read_csv(vtsmx_data, index_col="Date", infer_datetime_format=True, parse_dates=True)

# Sort DataFrame by Date Index
vtsmx_df.sort_index(inplace=True)
vtsmx_df.head()

,Close
Date,
2012-01-03,31.76
2012-01-04,31.75
2012-01-05,31.87
2012-01-06,31.81
2012-01-09,31.90


In [3]:
# Check Data Types
print(vtsmx_df.dtypes)

Close    float64
dtype: object


In [4]:
# Sample DataFrame so it contains closing for month ends
# Monthly Data will be used later as Inflation data is monthly
vtsmx_df = vtsmx_df.resample('M').last()
vtsmx_df

,Close
Date,
2012-01-31,32.88
2012-02-29,34.28
2012-03-31,35.19
2012-04-30,34.96
2012-05-31,32.78
...,...
2021-08-31,113.46
2021-09-30,108.06
2021-10-31,115.31


In [5]:
# Calculate Monthly Returns
vtsmx_df['Monthly Returns'] = vtsmx_df['Close'].pct_change()
vtsmx_df.head()

,Close,Monthly Returns
Date,,
2012-01-31,32.88,NaN
2012-02-29,34.28,0.042579
2012-03-31,35.19,0.026546
2012-04-30,34.96,-0.006536
2012-05-31,32.78,-0.062357


In [6]:
# Rename `Close` & 'Returns' Columns to be specific  
columns = ["VTSMX Close", "VTSMX Returns"]
vtsmx_df.columns = columns
vtsmx_df.head()

,VTSMX Close,VTSMX Returns
Date,,
2012-01-31,32.88,NaN
2012-02-29,34.28,0.042579
2012-03-31,35.19,0.026546
2012-04-30,34.96,-0.006536
2012-05-31,32.78,-0.062357


In [7]:
# Count nulls
vtsmx_df.isnull().sum()

VTSMX Close      0
VTSMX Returns    1
dtype: int64

In [8]:
# Drop nulls - Used the `dropna` function to drop whole records that have at least one null value
vtsmx_df.dropna(inplace=True)
# Check no more nulls
vtsmx_df.isnull().sum()

VTSMX Close      0
VTSMX Returns    0
dtype: int64

## VGTSX Data
Read the VGTSX csv and clean the data.


In [9]:
# Set the Path
vgtsx_data = Path("Resources/VGTSX_data.csv")

# Read the historic closing prices and create a new daily returns DataFrame from the data. 
vgtsx_df = pd.read_csv(vgtsx_data, index_col="Date", infer_datetime_format=True, parse_dates=True)

# Sort DataFrame by Date Index
vgtsx_df.sort_index(inplace=True)
vgtsx_df.head()

,Close
Date,
2017-01-03,14.82
2017-01-04,14.97
2017-01-05,15.12
2017-01-06,15.06
2017-01-09,15.02


In [10]:
# Check Data Types
print(vgtsx_df.dtypes)

Close    float64
dtype: object


In [11]:
# Sample DataFrame so it contains closing for month ends
# Monthly Data will be used later as Inflation data is monthly
vgtsx_df = vgtsx_df.resample('M').last()
vgtsx_df

,Close
Date,
2017-01-31,15.30
2017-02-28,15.53
2017-03-31,15.92
2017-04-30,16.27
2017-05-31,16.76
2017-06-30,16.67
2017-07-31,17.24
2017-08-31,17.35
2017-09-30,17.57


In [12]:
# Calculate Monthly Returns
vgtsx_df['Monthly Returns'] = vgtsx_df['Close'].pct_change()
vgtsx_df.head()

,Close,Monthly Returns
Date,,
2017-01-31,15.30,NaN
2017-02-28,15.53,0.015033
2017-03-31,15.92,0.025113
2017-04-30,16.27,0.021985
2017-05-31,16.76,0.030117


In [13]:
# Rename `Close` & 'Returns' Columns to be specific  
columns = ["VGTSX Close", "VGTSX Returns"]
vgtsx_df.columns = columns
vgtsx_df.head()

,VGTSX Close,VGTSX Returns
Date,,
2017-01-31,15.30,NaN
2017-02-28,15.53,0.015033
2017-03-31,15.92,0.025113
2017-04-30,16.27,0.021985
2017-05-31,16.76,0.030117


In [14]:
# Count nulls
vgtsx_df.isnull().sum()

VGTSX Close      0
VGTSX Returns    1
dtype: int64

In [15]:
# Drop nulls - Used the `dropna` function to drop whole records that have at least one null value
vgtsx_df.dropna(inplace=True)
# Check no more nulls
vgtsx_df.isnull().sum()

VGTSX Close      0
VGTSX Returns    0
dtype: int64

## VBMFX Data
Read the VBMFX csv and clean the data.


In [16]:
# Set the Path
vbmfx_data = Path("Resources/VBMFX_data.csv")

# Read the historic closing prices and create a new daily returns DataFrame from the data. 
vbmfx_df = pd.read_csv(vbmfx_data, index_col="Date", infer_datetime_format=True, parse_dates=True)

# Sort DataFrame by Date Index
vbmfx_df.sort_index(inplace=True)
vbmfx_df.head()

,Close
Date,
2017-01-03,10.65
2017-01-04,10.66
2017-01-05,10.71
2017-01-06,10.67
2017-01-09,10.70


In [17]:
# Check Data Types
print(vbmfx_df.dtypes)

Close    float64
dtype: object


In [18]:
# Sample DataFrame so it contains closing for month ends
# Monthly Data will be used later as Inflation data is monthly
vbmfx_df = vbmfx_df.resample('M').last()
vbmfx_df

,Close
Date,
2017-01-31,10.66
2017-02-28,10.71
2017-03-31,10.68
2017-04-30,10.74
2017-05-31,10.79
2017-06-30,10.77
2017-07-31,10.79
2017-08-31,10.86
2017-09-30,10.78


In [19]:
# Calculate Monthly Returns
vbmfx_df['Monthly Returns'] = vbmfx_df['Close'].pct_change()
vbmfx_df.head()

,Close,Monthly Returns
Date,,
2017-01-31,10.66,NaN
2017-02-28,10.71,0.004690
2017-03-31,10.68,-0.002801
2017-04-30,10.74,0.005618
2017-05-31,10.79,0.004655


In [20]:
# Rename `Close` & 'Returns' Columns to be specific  
columns = ["VBMFX Close", "VBMFX Returns"]
vbmfx_df.columns = columns
vbmfx_df.head()

,VBMFX Close,VBMFX Returns
Date,,
2017-01-31,10.66,NaN
2017-02-28,10.71,0.004690
2017-03-31,10.68,-0.002801
2017-04-30,10.74,0.005618
2017-05-31,10.79,0.004655


In [21]:
# Count nulls
vbmfx_df.isnull().sum()

VBMFX Close      0
VBMFX Returns    1
dtype: int64

In [22]:
# Drop nulls - Used the `dropna` function to drop whole records that have at least one null value
vbmfx_df.dropna(inplace=True)
# Check no more nulls
vbmfx_df.isnull().sum()

VBMFX Close      0
VBMFX Returns    0
dtype: int64

## PIGLX Data

Read the PIGLX csv and clean the data.

In [23]:
# Set the Path
piglx_data = Path("Resources/PIGLX_data.csv")

# Read the historic closing prices and create a new daily returns DataFrame from the data. 
piglx_df = pd.read_csv(piglx_data, index_col="Date", infer_datetime_format=True, parse_dates=True)

# Sort DataFrame by Date Index
piglx_df.sort_index(inplace=True)
piglx_df.head()

,Close
Date,
2017-01-03,8.89
2017-01-04,8.91
2017-01-05,8.98
2017-01-06,8.93
2017-01-09,8.96


In [24]:
# Check Data Types
print(piglx_df.dtypes)

Close    float64
dtype: object


In [25]:
# Sample DataFrame so it contains closing for month ends
# Monthly Data will be used later as Inflation data is monthly
piglx_df = piglx_df.resample('M').last()
piglx_df

,Close
Date,
2017-01-31,9.07
2017-02-28,9.13
2017-03-31,9.15
2017-04-30,9.23
2017-05-31,9.36
2017-06-30,9.36
2017-07-31,9.54
2017-08-31,9.64
2017-09-30,9.56


In [26]:
# Calculate Monthly Returns
piglx_df['Monthly Returns'] = piglx_df['Close'].pct_change()
piglx_df.head()

,Close,Monthly Returns
Date,,
2017-01-31,9.07,NaN
2017-02-28,9.13,0.006615
2017-03-31,9.15,0.002191
2017-04-30,9.23,0.008743
2017-05-31,9.36,0.014085


In [27]:
# Rename `Close` & 'Returns' Columns to be specific  
columns = ["PIGLX Close", "PIGLX Returns"]
piglx_df.columns = columns
piglx_df.head()

,PIGLX Close,PIGLX Returns
Date,,
2017-01-31,9.07,NaN
2017-02-28,9.13,0.006615
2017-03-31,9.15,0.002191
2017-04-30,9.23,0.008743
2017-05-31,9.36,0.014085


In [28]:
# Count nulls
piglx_df.isnull().sum()

PIGLX Close      0
PIGLX Returns    1
dtype: int64

In [29]:
# Drop nulls - Used the `dropna` function to drop whole records that have at least one null value
piglx_df.dropna(inplace=True)
# Check no more nulls
piglx_df.isnull().sum()

PIGLX Close      0
PIGLX Returns    0
dtype: int64

## Combine VGTSX, VGTSX, VBMFX, and PIGLX Returns

In [30]:
# Join all returns dataframes intp a single one. Drop columns so we have left only the ones with returns
all_returns_df = pd.concat([vgtsx_df, vtsmx_df, vbmfx_df, piglx_df], axis="columns", join="inner")
##all_returns_df

all_returns_df = all_returns_df.drop(all_returns_df.filter(regex='Close').columns, axis=1)
all_returns_df

,VGTSX Returns,VTSMX Returns,VBMFX Returns,PIGLX Returns
Date,,,,
2017-02-28,0.015033,0.037108,0.004690,0.006615
2017-03-31,0.025113,-0.003544,-0.002801,0.002191
2017-04-30,0.021985,0.010501,0.005618,0.008743
2017-05-31,0.030117,0.010057,0.004655,0.014085
2017-06-30,-0.005370,0.004812,-0.001854,0.000000
2017-07-31,0.034193,0.018662,0.001857,0.019231
2017-08-31,0.006381,0.001459,0.006487,0.010482
2017-09-30,0.012680,0.020398,-0.007366,-0.008299
2017-10-31,0.019351,0.021577,-0.000928,-0.005230


## Calculate the weighted returns for CONSERVATIVE portfolio

In [31]:
# Set weights
weights1 = [0.48, 0.12, 0.14, 0.26]

# Calculate portfolio return
conservative_df = all_returns_df.dot(weights1)

# Display sample data
conservative_df


Date
2017-02-28    0.014045
2017-03-31    0.011806
2017-04-30    0.014873
2017-05-31    0.019977
2017-06-30   -0.002260
2017-07-31    0.023912
2017-08-31    0.006871
2017-09-30    0.005345
2017-10-31    0.010388
2017-11-30    0.009072
2017-12-31    0.006218
2018-01-31    0.035992
2018-02-28   -0.035105
2018-03-31   -0.005426
2018-04-30   -0.001614
2018-05-31   -0.009153
2018-06-30   -0.016612
2018-07-31    0.017033
2018-08-31   -0.008782
2018-09-30   -0.001683
2018-10-31   -0.055385
2018-11-30    0.007770
2018-12-31   -0.044493
2019-01-31    0.053086
2019-02-28    0.011748
2019-03-31    0.004944
2019-04-30    0.018046
2019-05-31   -0.032124
2019-06-30    0.035529
2019-07-31   -0.010207
2019-08-31   -0.010114
2019-09-30    0.009309
2019-10-31    0.021064
2019-11-30    0.007167
2019-12-31    0.021904
2020-01-31   -0.013001
2020-02-29   -0.042795
2020-03-31   -0.116384
2020-04-30    0.064309
2020-05-31    0.035468
2020-06-30    0.025319
2020-07-31    0.037168
2020-08-31    0.032140
2020-0

## Calculate the weighted returns for GROWTH portfolio

In [32]:
# Set weights
weights2 = [0.16, 0.04, 0.28, 0.52]

# Calculate portfolio return
growth_df = all_returns_df.dot(weights2)

# Display sample data
growth_df



Date
2017-02-28    0.008643
2017-03-31    0.004231
2017-04-30    0.010057
2017-05-31    0.013848
2017-06-30   -0.001186
2017-07-31    0.016737
2017-08-31    0.008346
2017-09-30   -0.003533
2017-10-31    0.000980
2017-11-30    0.005803
2017-12-31    0.002959
2018-01-31    0.016632
2018-02-28   -0.021657
2018-03-31    0.000886
2018-04-30   -0.010612
2018-05-31   -0.008554
2018-06-30   -0.008768
2018-07-31    0.007565
2018-08-31   -0.006899
2018-09-30   -0.001410
2018-10-31   -0.029146
2018-11-30    0.001361
2018-12-31   -0.022720
2019-01-31    0.028355
2019-02-28    0.002768
2019-03-31    0.004681
2019-04-30    0.006063
2019-05-31   -0.006987
2019-06-30    0.020776
2019-07-31   -0.008170
2019-08-31    0.002606
2019-09-30   -0.001180
2019-10-31    0.010899
2019-11-30   -0.002084
2019-12-31    0.014215
2020-01-31    0.000577
2020-02-29   -0.015499
2020-03-31   -0.073817
2020-04-30    0.037434
2020-05-31    0.021746
2020-06-30    0.017089
2020-07-31    0.030228
2020-08-31    0.015482
2020-0

## Calculate the weighted returns for MODERATE portfolio

In [ ]:
# Set weights
weights3 = [0.32, 0.08, 0.21, 0.39]

# Calculate portfolio return
moderate_df = all_returns_df.dot(weights3)

# Display sample data
moderate_df

